# [0] Imports

In [194]:
import Pkg;

using LinearAlgebra, Random, Gurobi, JuMP, Distributions, Plots, LazySets

using Dates

In [195]:
LOW_JOBS = 10
HIGH_JOBS = 25;

In [196]:
times_test = []
OBJECTIVES = [] # record the objective
ROUTES = []
;

In [197]:
for job in LOW_JOBS:HIGH_JOBS
    n_jobs = job
    n_vehicles_jobs = n_jobs
    n_vehicles_coverage = 10
    T = 2 * n_jobs
    min_duration = 2
    max_duration = 6
    speed = 1000/6
    coverage_distance = 50
    size = 500
    step = 50;

    Random.seed!(1234)

    function create_cluster_sizes(jobs_total)
        Random.seed!(1234)
        jobs_created = 0
        cluster = []
        min_jobs_allowed = n_jobs >= 15 ? 3 : 2
            target_clusters = 5
            while jobs_created != jobs_total
                num_to_add = rand(min(min_jobs_allowed, jobs_total - jobs_created) : 
                    min(Int((n_jobs/target_clusters)÷1), jobs_total - jobs_created))
                jobs_created += num_to_add
                push!(cluster, num_to_add)
            end

        return cluster
    end;

    time_windows = []
    locations = rand(Uniform(0,size), 1, 2)
    work_load = []
    cluster = create_cluster_sizes(n_jobs);

    function create_time_windows_and_work_load(cluster_sizes, locations)
        Random.seed!(1234)
        locations = rand(Uniform(0,size), 1, 2)
        for size_c in cluster
            first = rand(Uniform(0,size), 1, 2)
            locations = vcat(locations, first)

            job_begins = rand(2:10)
            job_finish = rand((job_begins+min_duration):(job_begins+max_duration))
            push!(time_windows, [job_begins, job_finish])

            time_work = rand(min_duration:max(min_duration, job_finish - job_begins))
            push!(work_load, time_work)

            for neighbour in 1:(size_c-1)
                new_x = rand(Uniform(max(0,first[1]-20), min(first[1]+20, size)), 1, 1)
                new_y = rand(Uniform(max(0,first[2]-20), min(first[2]+20, size)), 1, 1)
                new = hcat(new_x, new_y)
                locations = vcat(locations, new)

                job_begins = rand(job_finish:min(T-min_duration-2, job_finish + 6))
                job_finish = rand((job_begins+min_duration):(min(job_begins+max_duration, T-2)))
                push!(time_windows, [job_begins, job_finish])

                time_work = rand(min_duration:min(max_duration, job_finish-job_begins))
                push!(work_load, time_work)
            end
        end

        return [time_windows, work_load, locations]
    end;

    time_windows, work_load, locations = create_time_windows_and_work_load(cluster, locations)
    distances = [LinearAlgebra.norm(locations[i, :] .- locations[j, :]) for i=1:n_jobs+1, j = 1:n_jobs+1];

    TIMING = datetime2unix(now())
    routes = []

    for n in 1:n_jobs
        job_route = []
        dist = distances[1, n+1]

        min_t = Int(floor(time_windows[n][1] - dist/speed))
        max_t = Int(ceil(time_windows[n][2] + dist/speed))

        push!(job_route, [[0, min_t], [n, time_windows[n][1]], dist])

        for t in time_windows[n][1]:(time_windows[n][2]-1)
            push!(job_route, [[n, t], [n, t+1], 0])
        end

        push!(job_route, [[n, time_windows[n][2]], [n_jobs+1, max_t],  dist])
        push!(routes, job_route)

    end

    function compute_cost(route)
        cost = 0
        for entity in route
            cost += entity[3]
        end
        return cost
    end;

    C = []
    for i in 1:length(routes)
        push!(C, compute_cost(routes[i]))
    end

    function compute_delta(routes)
        Q = length(routes)
        delta = [[[0.0 for q in 1:Q] for t in 1:T] for i in 1:n_jobs]
        for rindex in 1:Q
            route = routes[rindex]
            for arc in route
                loc1, time1 = arc[1]
                loc2, time2 = arc[2]
                if (loc1 != 0) & (loc1 != n_jobs + 1)
                    delta[loc1][time1][rindex] = 1
                end

                if (loc2 != 0) & (loc2 != n_jobs + 1)
                    delta[loc2][time2][rindex] = 1
                end
            end
        end
        return delta
    end;

    delta = compute_delta(routes);

    function compute_u(routes)
        Q = length(routes)
        u = [[0 for q in 1:Q] for i in 1:n_jobs]

        for rindex in 1:Q
            route = routes[rindex]
            for arc in route
                loc1 = arc[1][1]
                loc2 = arc[2][1]
                if (1 <= loc1 <= n_jobs)
                    u[loc1][rindex] = 1
                end
                if (1 <= loc2 <= n_jobs)
                    u[loc2][rindex] = 1
                end
            end
        end
        return u
    end;

    u = compute_u(routes);

    function sp_lsa(n, travel_distance, travel_time, windows, load, rho_v, pi_v, mu_v)
        N = [[1]] 
        Times = [ [0] ] 
        R = [rho_v]  
        L = [1]  

        current_state = 1
        total_state = 1

        #=
        NOTE: For pi_v, u_v, mu_v, and delta_v, there is no defined value
        for the actual depot. We will be using -1's for indexing quite liberally,
        as in our loop, 1 = n+2 = depot, when in 'reality', this should be 0 = n+!.

        This is also the case for variables windows and load. This happened last LSA too.
        =#
        while 1==1

            # STEP 1: Check whether we want to move on to checking the next path
            if (L[current_state] == n+2)
                current_state += 1
                if current_state > total_state
                    # print(current_state, " ", total_state, " nothing more to check")
                    break
                else
                    continue
                end
            end

            for i in 2:(n+2)

                if ~(i in N[current_state])
                    cur_time = last(Times[current_state])
                    dist_nec = travel_time[L[current_state], i]
                    old_job_time_nec = L[current_state] > 1 ? load[L[current_state]-1] : 0
                    new_job_time_nec = load[i-1]
                    new_window_start, new_window_close = windows[i-1]

                    if L[current_state] != 1
                        if cur_time + dist_nec + old_job_time_nec + new_job_time_nec > new_window_close
                            continue
                        end
                    end

                    reach_new_node_time = max(cur_time + dist_nec + old_job_time_nec, new_window_start)

                    # SECOND CHECK
                    add_segdist = travel_distance[L[current_state], i]
                    subtract_pi = i < n+2 ? -pi_v[i-1] : 0
                    subtract_mu = 0.0
                    if i < n+2
                        for applicable_time in reach_new_node_time:reach_new_node_time+new_job_time_nec
                            subtract_mu -= mu_v[Int(i-1), Int(applicable_time)]
                        end
                    end

                    proposed_cost = R[current_state] + add_segdist + subtract_pi + subtract_mu

                    # STEP 2-3: Add the path to our N.
                    push!(N, copy(N[current_state]))
                    push!(N[total_state+1], i)

                    new_times_array = copy(Times[current_state])
                    push!(new_times_array, reach_new_node_time)
                    push!(Times, new_times_array)

                    push!(R, proposed_cost)

                    push!(L, i)

                    total_state += 1

                end
            end

            current_state += 1 # concluded iteration for that state in mind. Now next one

            if current_state == total_state
                # print(current_state, " ", total_state, " came from bottom")
                break
            end     
        end

        return N, R, Times
    end;

    distances_label = deepcopy(distances)
    distances_label = hcat(distances_label, distances_label[:, 1])
    distances_label = vcat(distances_label, collect(push!(distances_label[:, 1], 0)'));

    travel_times = ceil.(distances_label / speed);

    windows_label = deepcopy(time_windows)
    push!(windows_label, [0, 100]);

    load_label = deepcopy(work_load)
    push!(load_label, 0);

    function extract_best_route(sub_paths, sub_rc, sub_times, n_jobs)

        best_rc = Inf
        best_index = 0

        for i in 1:length(sub_paths)
            if sub_rc[i] < best_rc && last(sub_paths[i]) == n_jobs + 2
                best_rc = sub_rc[i]
                best_index = i
            end
        end

        return sub_paths[best_index], sub_rc[best_index], sub_times[best_index]
    end;

    function generate_full_time_info(best_route, best_times)
        best_route_info = []
        for locindex in 1:length(best_route)-1

            cur_loc = best_route[locindex]
            cur_time = best_times[locindex]
            time_to_start_moving = best_times[locindex]

            #stationary data comes first.
            if 1 < locindex < n_jobs + 2
                cur_job_num = cur_loc - 1 # delete padding!

                for time in cur_time:cur_time+work_load[cur_job_num]-1
                    route_detail = [ [cur_loc-1, time], [cur_loc-1, time+1], 0]
                    push!(best_route_info, route_detail)
                end

                time_to_start_moving += work_load[cur_job_num]
            end

            new_loc = best_route[locindex+1]
            #then if we move, we move.
            route_detail = [ [cur_loc-1, time_to_start_moving], [new_loc-1, best_times[locindex+1]], distances_label[cur_loc, new_loc]]
            push!(best_route_info, route_detail)
        end

        return best_route_info
    end;

    modelcg = Model(Gurobi.Optimizer);
    set_optimizer_attribute(modelcg, "OutputFlag", 0);

    @variable(modelcg, 0 <= y_s[1:n_jobs, 1:T] <= 1) # should be Bin
    @variable(modelcg, 0 <= y_e[1:n_jobs, 1:T] <= 1); # should be Bin

    @constraint(modelcg, start[i in 1:n_jobs, t in 1:(T-1)], y_s[i,t] <= y_s[i,t+1])
    @constraint(modelcg, ends[i in 1:n_jobs, t in 1:(T-1)], y_e[i,t] <= y_e[i,t+1])

    @constraint(modelcg, window_start[i in 1:n_jobs, t in 1:(time_windows[i][1]-1)], y_s[i,t] == 0)
    @constraint(modelcg, window_ends[i in 1:n_jobs, t in time_windows[i][2]:T], y_e[i,t] == 1)

    @constraint(modelcg, duration[i in 1:n_jobs], sum(y_s[i,t] - y_e[i,t] for t in 1:T) >= work_load[i]);

    @constraint(modelcg, coverage[i in 1:n_jobs, t in 1:T], y_s[i, t] - y_e[i, t] <= 1);

    keep_going = true

    num_iters = 0
    num_routes_at_a_time = 1
    z_values = []
    OBJ = 0

    while keep_going
        num_iters += 1
        Q = length(routes)

        unregister(modelcg, :z)
        @variable(modelcg, 0 <= z[1:Q] <= 1) # should be Bin

        unregister(modelcg, :unique)
        @constraint(modelcg, unique[i in 1:n_jobs], sum(u[i][q] * z[q] for q in 1:Q) == 1);

        unregister(modelcg, :driver)
        @constraint(modelcg, driver, sum(z[q] for q in 1:Q) <= n_vehicles_jobs)

        unregister(modelcg, :work)
        @constraint(modelcg, work[i in 1:n_jobs, t in 1:(T-1)], 
            y_s[i, t] - y_e[i, t] <= sum(z[q] * delta[i][t][q] for q in 1:Q));

        @objective(modelcg, Min, sum(compute_cost(routes[q]) * z[q] for q in 1:Q));

        optimize!(modelcg)
        OBJ = objective_value(modelcg)

        push!(z_values, round.(value.(z), digits = 3))

        pi_values = dual.(unique)
        rho_value = dual.(driver)
        mu_values = dual.(work);

        sub_paths, sub_rc, sub_times = sp_lsa(n_jobs, distances_label, travel_times, windows_label, load_label, rho_value, pi_values, mu_values);

        best_route, best_rc, best_times = extract_best_route(sub_paths, sub_rc, sub_times, n_jobs)

        if best_rc > -1e-6
            keep_going = false
            push!(OBJECTIVES, OBJ)
            push!(times_test, datetime2unix(now()) - TIMING)
            break
        end

        push!(routes, generate_full_time_info(best_route, best_times))

        u = compute_u(routes)
        delta = compute_delta(routes);

        C = []
        for i in 1:length(routes)
            push!(C, compute_cost(routes[i]))
        end;
    end
    
    route_descs = []
    for i in n_jobs+1:length(routes)
        if last(z_values)[i] > 0.1
            nodes_visited = []
            for entity in routes[i]
                place = entity[1][1]
                if ~(place in nodes_visited)
                    push!(nodes_visited, place)
                end
            end
            push!(nodes_visited, n_jobs+1)
            push!(route_descs, nodes_visited)
        end
    end
    push!(ROUTES, route_descs)
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04

In [198]:
times_test

20-element Vector{Any}:
  1.2000000476837158
  0.03500008583068848
  0.04399991035461426
  0.04700016975402832
  0.06599998474121094
  0.12100005149841309
  0.17200016975402832
  0.21499991416931152
  0.25
  0.46899986267089844
  1.2360000610351562
  1.7609999179840088
  1.6460001468658447
  2.8299999237060547
  3.1440000534057617
  6.924999952316284
  9.480000019073486
  8.07200002670288
 14.559999942779541
 17.147000074386597

In [199]:
ROUTES[3]

6-element Vector{Any}:
 Any[0, 9, 10, 13]
 Any[0, 7, 8, 13]
 Any[0, 3, 4, 13]
 Any[0, 5, 6, 13]
 Any[0, 1, 2, 13]
 Any[0, 11, 12, 13]

In [200]:
for i in LOW_JOBS:HIGH_JOBS
    println(round(times_test[i-LOW_JOBS+1], digits=2))
end

1.2
0.04
0.04
0.05
0.07
0.12
0.17
0.21
0.25
0.47
1.24
1.76
1.65
2.83
3.14
6.92
9.48
8.07
14.56
17.15


In [201]:
for i in LOW_JOBS:HIGH_JOBS
    println("i: ", i, " OBJECTIVE: ", OBJECTIVES[i-LOW_JOBS+1])
end

i: 10 OBJECTIVE: 2262.5866644782222
i: 11 OBJECTIVE: 2658.537247886904
i: 12 OBJECTIVE: 2717.4076077549953
i: 13 OBJECTIVE: 2987.198492629768
i: 14 OBJECTIVE: 3035.4229541948375
i: 15 OBJECTIVE: 2539.819016424984
i: 16 OBJECTIVE: 2566.0534756445545
i: 17 OBJECTIVE: 3012.9949837073295
i: 18 OBJECTIVE: 3014.1635408968973
i: 19 OBJECTIVE: 3036.732890636053
i: 20 OBJECTIVE: 3028.438121554299
i: 21 OBJECTIVE: 3257.98557769283
i: 22 OBJECTIVE: 3364.3580682416987
i: 23 OBJECTIVE: 3399.8770078511234
i: 24 OBJECTIVE: 3445.0403841209595
i: 25 OBJECTIVE: 2845.6632169349186
i: 26 OBJECTIVE: 2856.061688153195
i: 27 OBJECTIVE: 2867.9501617736605
i: 28 OBJECTIVE: 2933.9539820757536
i: 29 OBJECTIVE: 3019.589345702565
